---

## Random Forest

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
X_train = series_train.drop(columns=['sales', 'date','diff','Train_Forecast', 'Train_Forecast_prophet', 'train_Forecast_LSTM','train_Forecast_XGB'])  # Exclure les colonnes inutiles pour l'entraînement
y_train = series_train['sales']

X_test = series_test.drop(columns=['sales', 'date','Test_Forecast','Test_Forecast_prophet', 'test_Forecast_LSTM', 'test_Forecast_XGB']) # Exclure les colonnes inutiles pour les prédictions
y_test = series_test['sales']

# girdsearch


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],         # Nombre d'arbres dans la forêt
    'max_depth': [5, 10, 20],              # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],       # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]          # Nombre minimum d'échantillons dans une feuille
}

grid_search_RF = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,                                  # Validation croisée (5-fold)
    scoring='neg_mean_squared_error',      
    verbose=1,                             
    n_jobs=-1                              # Utilisation maximale des CPU disponibles
)


grid_search_RF.fit(X_train, y_train)

print("Meilleurs paramètres trouvés :")
print(grid_search_RF.best_params_)


best_RF_model = grid_search_RF.best_estimator_

# --predictions train--

y_train_pred = best_RF_model.predict(X_train)
series_train['train_Forecast_RF'] = y_train_pred

# --predictions test--

y_pred = best_RF_model.predict(X_test)
series_test['test_Forecast_RF'] = y_pred


Fitting 5 folds for each of 81 candidates, totalling 405 fits


Meilleurs paramètres trouvés :
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
RMSE avec le meilleur modèle Random Forest : 364096.35630466416


In [5]:
X_train

,onpromotion,oil_price,holiday
0,0,93.080000,0
1,0,93.810000,0
2,0,93.600000,0
3,0,93.767547,1
4,0,94.102453,0
...,...,...,...
1079,2582,36.675088,0
1080,1316,36.360000,0
1081,3848,37.880000,0
1082,9068,36.590000,0


In [6]:
voirpour RF puis faire un code qui fait pred sur chauqe magasin puis agggreer. peu tetre u code qui teste lstm prophet sur chacun et prend le meilelleurs des deu sur chacun . on aggrege. faire pareil sur cluster. 

SyntaxError: invalid syntax (231600910.py, line 1)

In [ ]:

# %%
# Prédictions sur l'ensemble de test
y_pred = best_RF_model.predict(X_test)

# Évaluation
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE avec le meilleur modèle Random Forest : {rmse}")

# %%
# Ajout des prédictions au DataFrame de test
series_test['Predictions'] = y_pred

# %%
# Agrégation des prédictions par magasin
if 'store' in series_test.columns:
    aggregated_predictions = series_test.groupby('store')[['sales', 'Predictions']].sum()
    print(aggregated_predictions.head())
else:
    print("La colonne 'store' est absente, impossible d'agréger par magasin.")

# %%
# Sauvegarder les prédictions
series_test.to_csv("series_test_with_predictions.csv", index=False)


#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)